<a href="https://colab.research.google.com/github/Snehallaldas/Gemma3-270M/blob/main/Gemma_3_270M.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Step 1: Import the Dataset

In [11]:
!pip install datasets

In [12]:
from datasets import load_dataset

ds = load_dataset("roneneldan/TinyStories")

In [13]:
!pip install tiktoken
import tiktoken
import os
import numpy as np
from tqdm.auto import tqdm

enc = tiktoken.get_encoding("gpt2")

def process(example):
  ids = enc.encode_ordinary(example['text'])  # encode_ordinary ignores the special tokens
  out = {'ids': ids, 'len': len(ids)}
  return out

if not os.path.exists("train.bin"):
  tokenized = ds.map(
      process,
      remove_columns=['text'],
      desc="tokenizing the splits",
      num_proc=8,
      )
  # concatinate all the ids in each dataset into one large file we can use for training
  for split,dset in tokenized.items():
    arr_len = np.sum(dset['len'], dtype=np.uint64)
    filename = f'{split}.bin'
    dtype = np.uint16
    arr = np.memmap(filename, dtype=dtype, mode='w+', shape=(arr_len,))
    total_batches = 1024

    idx = 0
    for batch_idx in tqdm(range(total_batches), desc=f'writing {filename}'):
      batch = dset.shard(num_shards=total_batches, index=batch_idx, contiguous=True).with_format('numpy')
      arr_batch = np.concatenate(batch['ids'])
      arr[idx : idx + len(arr_batch)] = arr_batch
      idx += len(arr_batch)
    arr.flush()

## Create Inout and output Batches

In [15]:
 def get_batch(split):
  if split == 'train':
    data = np.memmap('train.bin', dtype=np.uint16, mode='r')
  else:
    data = np.memmap('validation.bin', dtype=np.uint16, mode='r')
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([torch.from_numpy((data[i:i+block_size]).astype(np.int64)) for i in ix])
  y = torch.stack([torch.from_numpy((data[i+1:i+1+block_size]).astype(np.int64)) for i in ix])
  if device_type == 'cuda':
    x, y = x.pin_memory().to(device, non_blocking=True), y.pin_memory().to(device, non_blocking=True)
  else:
    x,y = x.to(device), y.to(device)
  return x, y


## Define the